# Performance dataframes

In [2]:
import os
import re
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

/Users/andersvandvik/Repositories/masters-thesis/venv/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## General parameters

In [3]:
nbr_sims = 5

instance_key = 'instance'
best_obj_key = 'best obj'
worst_obj_key = 'worst obj'
avg_obj_key = 'obj'
max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'
max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_iter_key = 'iter'
set_part_key = 'sp improv'

gap_key = 'gap'
preprocess_key = 'preproc'
model_key = 'model time'
variables_key = 'variables'


project_path = os.path.dirname(os.path.abspath('.'))
directory_path_alns = '/output/solstorm/alns/performance/'
directory_path_exact = '/output/solstorm/arcflow/performance/'

generate_df = True
run_number = 'third'

## Functions

In [4]:
def map_instance_to_data_alns(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
        
            obj = history_json['best_objective']
            time = history_json['runtime']
            it = history_json['number_of_iterations']
            set_part_improv = history_json['number_of_improvements_by_set_partitioning']

            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]

                if obj < data[0]:
                    data[0] = obj
                if obj > data[1]:
                    data[1] = obj
                if time > data[3]:
                    data[3] = time
                if time < data[4]:
                    data[4] = time
                if it > data[6]:
                    data[6] = it
                if it < data[7]:
                    data[7] = it

                data[2] += obj
                data[5] += time
                data[8] += it
                data[9] += set_part_improv
                data[10] += 1
            else:
                instance_to_data[instance_name] = [obj, obj, obj, time, time, time, it, it, it, set_part_improv, 1]
    return instance_to_data

def generate_run_df_alns(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_alns(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               best_obj_key, worst_obj_key, avg_obj_key, 
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        best_objective = data[0]
        worst_objective = data[1]
        avg_objective = data[2] / nbr_sims
        max_time = data[3]
        min_time = data[4]
        avg_time = data[5] / nbr_sims
        max_iter = data[6]
        min_iter = data[7]
        avg_iter = data[8] / nbr_sims
        avg_set_part_improv = data[9] / nbr_sims
        
        if data[10] != nbr_sims:
            print(f'{instance} HAS LESS THAN FIVE SIMULATIONS!')
        
        row = pd.Series({instance_key: instance, 
                         best_obj_key: best_objective,
                         worst_obj_key: worst_objective,
                         avg_obj_key: avg_objective, 
                         max_time_key: max_time,
                         min_time_key: min_time,
                         avg_time_key: avg_time, 
                         max_iter_key: max_iter,
                         min_iter_key: min_iter,
                         avg_iter_key: avg_iter,
                         set_part_key: avg_set_part_improv})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def map_instance_to_data_exact(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        with open(run_path + file_name) as file:
            exact_json = json.load(file)

        obj = exact_json['objective']['objective_bound']
        gap = exact_json['objective']['optimality_gap']
        model_runtime = exact_json['runtime']['model_runtime']
        preprocess_runtime = exact_json['runtime']['preprocess_runtime']
        variables = exact_json['variables']['number_of_variables']
        
        if instance_name in instance_to_data:
            print('Multiple versions of same instance!')
        
        instance_to_data[instance_name] = [obj, gap, model_runtime, preprocess_runtime, variables]
    
    return instance_to_data

def generate_run_df_exact(run_name):
    run_path = project_path + directory_path_exact + run_name
    instance_to_data = map_instance_to_data_exact(run_path)

    df = pd.DataFrame(columns=[instance_key, avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    for instance in instance_to_data:
        data = instance_to_data[instance]
        obj = data[0]
        gap = data[1]
        preprocess_runtime = data[2]
        model_runtime = data[3]
        variables = data[4]
        row = pd.Series({instance_key: instance,
                         avg_obj_key: obj,
                         gap_key: gap,
                         preprocess_key: preprocess_runtime,
                         model_key: model_runtime,
                         variables_key: variables})
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(),
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, max_time_key, min_time_key, max_iter_key, min_iter_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(5, len(df_total.columns), 5)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def aggregate_df_by_instance_group_alns(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        best_obj = row[best_obj_key]
        worst_obj = row[worst_obj_key]
        avg_obj = row[avg_obj_key]
        max_time = row[max_time_key]
        min_time = row[min_time_key]
        avg_time = row[avg_time_key]
        max_iter = row[max_iter_key]
        min_iter = row[min_iter_key]
        avg_iter = row[avg_iter_key]
        avg_set_part_improv = row[set_part_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += best_obj
            data[1] += worst_obj
            data[2] += avg_obj
            data[3] += max_time
            data[4] += min_time
            data[5] += avg_time
            data[6] += max_iter
            data[7] += min_iter
            data[8] += avg_iter
            data[9] += avg_set_part_improv
            data[10] += 1  # Number of times encountered instance size
        else:
            data = [best_obj, worst_obj, avg_obj, max_time, min_time, avg_time, max_iter, min_iter, avg_iter, avg_set_part_improv, 1]
            instance_size_to_data[instance_size] = data

    df = pd.DataFrame(columns=['instance_group', best_obj_key, worst_obj_key, avg_obj_key, max_time_key, min_time_key,
                               avg_time_key, max_iter_key, min_iter_key, avg_iter_key, set_part_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[10]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         best_obj_key: data[0] / nbr_sims,
                         worst_obj_key: data[1] / nbr_sims,
                         avg_obj_key: data[2] / nbr_sims, 
                         max_time_key: data[3] / nbr_sims,
                         min_time_key: data[4] / nbr_sims,
                         avg_time_key: data[5] / nbr_sims, 
                         max_iter_key: data[6] / nbr_sims,
                         min_iter_key: data[7] / nbr_sims,
                         avg_iter_key: data[8] / nbr_sims,
                         set_part_key: data[9] / nbr_sims})
    
        df = df.append(row, ignore_index=True)
        
    mean_row = pd.Series({'instance_group': 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def aggregate_df_by_instance_group_exact(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        obj = row[avg_obj_key]
        gap = row[gap_key]
        preprocess_runtime = row[preprocess_key]
        model_runtime = row[model_key]
        variables = row[variables_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += obj
            data[1] += gap
            data[2] += preprocess_runtime
            data[3] += model_runtime
            data[4] += variables
            data[5] += 1 # Number of times encountered instance size
        else:
            instance_size_to_data[instance_size] = [obj, gap, preprocess_runtime, model_runtime, variables, 1]

    df = pd.DataFrame(columns=['instance_group', avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[5]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         avg_obj_key: data[0] / nbr_sims,
                         gap_key: data[1] / nbr_sims,
                         preprocess_key: data[2] / nbr_sims, 
                         model_key: data[3] / nbr_sims,
                         variables_key: data[4] / nbr_sims})
    
        df = df.append(row, ignore_index=True)

        
    mean_row = pd.Series({'instance_group': 'Mean values', 
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(), 
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def load_df(file_name):
    run_df = pd.read_pickle(f'dataframes/performance/{file_name}')
    # run_df = sort_df(run_df, sort_column)
    return run_df

## ALNS baseline

In [23]:
if generate_df:
    run_baseline_name = f'{run_number}/baseline/'
    run_baseline_df = generate_run_df_alns(run_baseline_name)
    run_baseline_agg_df = aggregate_df_by_instance_group_alns(run_baseline_df)

    run_baseline_file_name = f'dataframes/performance/baseline.pkl'
    run_baseline_agg_file_name = f'dataframes/performance/baseline_agg.pkl'
    run_baseline_df.to_pickle(run_baseline_file_name)
    run_baseline_agg_df.to_pickle(run_baseline_agg_file_name)

In [24]:
run_baseline_df = load_df('baseline.pkl')
run_baseline_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.8,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.8,0.9,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.3,1.2,1.2,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.1,1.1,1.1,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,1.9,1.7,1.8,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,1.7,1.5,1.6,5000,5000,5000.0,0.0
7,7-8-1-3,2080.2,2080.2,2080.2,2.0,1.8,1.9,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.8,4.1,4.3,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,2.7,2.1,2.3,5000,5000,5000.0,0.0


In [25]:
run_baseline_agg_df = load_df('baseline_agg.pkl')
run_baseline_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,0.9,0.9,5000.0,5000.0,5000.0,0.0
1,7,2095.0,2095.0,2095.0,2.6,2.2,2.4,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,5.0,4.0,4.5,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,19.7,17.7,18.7,5000.0,5000.0,5000.0,0.0
4,13,3970.5,3970.5,3970.5,33.7,30.2,32.2,5000.0,5000.0,5000.0,0.0
5,15,8267.2,8430.7,8378.3,41.3,34.2,37.8,5000.0,5000.0,5000.0,0.0
6,17,4953.6,5017.1,4977.9,62.5,56.0,59.2,5000.0,5000.0,5000.0,0.0
7,19,4909.5,5169.9,5046.6,78.3,65.5,71.7,5000.0,5000.0,5000.0,0.0
8,21,8118.4,9350.8,9065.7,95.0,76.4,86.0,5000.0,5000.0,5000.0,0.0
9,23,5647.7,5980.6,5820.2,137.8,123.6,130.1,5000.0,5000.0,5000.0,0.0


## Sequential ALNS

In [ ]:
if generate_df:
    run_sequential_name = 'first/sequential/'
    run_sequential_df = generate_run_df_alns(run_sequential_name)
    run_sequential_agg_df = aggregate_df_by_instance_group_alns(run_sequential_df)

    run_sequential_file_name = f'dataframes/performance/sequential.pkl'
    run_sequential_agg_file_name = f'dataframes/performance/sequential_agg.pkl'
    run_sequential_df.to_pickle(run_sequential_file_name)
    run_sequential_agg_df.to_pickle(run_sequential_agg_file_name)

In [ ]:
run_sequential_df = load_df('sequential.pkl')
run_sequential_df

In [ ]:
run_sequential_agg_df = load_df('sequential_agg.pkl')
run_sequential_agg_df

## LNS

In [15]:
if generate_df:
    run_lns_name = f'{run_number}/lns/'
    run_lns_df = generate_run_df_alns(run_lns_name)
    run_lns_agg_df = aggregate_df_by_instance_group_alns(run_lns_df)

    run_lns_file_name = f'dataframes/performance/lns.pkl'
    run_lns_agg_file_name = f'dataframes/performance/lns_agg.pkl'
    run_lns_df.to_pickle(run_lns_file_name)
    run_lns_agg_df.to_pickle(run_lns_agg_file_name)

In [16]:
run_lns_df = load_df('lns.pkl')
run_lns_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.8,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.8,0.8,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.3,1.2,1.3,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.1,1.1,1.1,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,1.8,1.7,1.7,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,1.7,1.5,1.6,5000,5000,5000.0,0.0
7,7-8-1-3,2080.2,2080.2,2080.2,2.0,1.8,1.9,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.5,4.1,4.3,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,2.7,2.0,2.3,5000,5000,5000.0,0.0


In [17]:
run_lns_agg_df = load_df('lns_agg.pkl')
run_lns_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,0.9,0.9,5000.0,5000.0,5000.0,0.0
1,7,2095.0,2095.0,2095.0,2.5,2.2,2.4,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,5.1,4.1,4.5,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,19.7,17.3,18.4,5000.0,5000.0,5000.0,0.0
4,13,3970.5,3970.5,3970.5,34.3,29.8,31.9,5000.0,5000.0,5000.0,0.0
5,15,8375.4,8431.3,8388.6,40.3,33.1,37.3,5000.0,5000.0,5000.0,0.0
6,17,4966.2,4975.9,4970.8,60.7,51.6,56.5,5000.0,5000.0,5000.0,0.0
7,19,4984.8,5201.5,5101.4,73.9,63.8,67.8,5000.0,5000.0,5000.0,0.0
8,21,7524.0,9351.4,8925.7,89.8,75.8,82.7,5000.0,5000.0,5000.0,0.0
9,23,5553.4,6001.5,5824.3,137.3,118.8,127.2,5000.0,5000.0,5000.0,0.0


## ALNS + local search

In [18]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    run_ls_df = generate_run_df_alns(run_ls_name)
    run_ls_agg_df = aggregate_df_by_instance_group_alns(run_ls_df)

    run_ls_file_name = f'dataframes/performance/ls.pkl'
    run_ls_agg_file_name = f'dataframes/performance/ls_agg.pkl'
    run_ls_df.to_pickle(run_ls_file_name)
    run_ls_agg_df.to_pickle(run_ls_agg_file_name)

In [19]:
run_ls_df = load_df('ls.pkl')
run_ls_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,1.5,1.4,1.4,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,1.3,1.3,1.3,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,1.5,1.4,1.5,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,2.2,2.1,2.1,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.8,1.7,1.8,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,4.3,4.2,4.3,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,3.9,3.3,3.7,5000,5000,5000.0,0.0
7,7-8-1-3,2080.2,2080.2,2080.2,4.7,4.3,4.5,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,7.2,6.7,7.0,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,5.1,4.9,5.0,5000,5000,5000.0,0.0


In [20]:
run_ls_agg_df = load_df('ls_agg.pkl')
run_ls_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.7,1.6,1.6,5000.0,5000.0,5000.0,0.0
1,7,2095.0,2095.0,2095.0,5.0,4.7,4.9,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,8.9,8.0,8.5,5000.0,5000.0,5000.0,0.0
3,11,3516.3,3517.5,3517.3,54.2,49.7,51.6,5000.0,5000.0,5000.0,0.0
4,13,3644.6,3984.8,3847.8,92.9,71.6,85.1,5000.0,5000.0,5000.0,0.0
5,15,6704.7,8389.0,8040.1,101.0,79.3,92.4,5000.0,5000.0,5000.0,0.0
6,17,4547.3,4939.1,4831.2,200.9,146.5,176.8,5000.0,5000.0,5000.0,0.0
7,19,4607.6,5084.5,4866.8,250.6,182.9,211.7,5000.0,5000.0,5000.0,0.0
8,21,7412.9,9329.7,8835.5,220.0,151.3,198.8,5000.0,5000.0,5000.0,0.0
9,23,5121.4,5747.6,5484.9,441.8,313.3,378.6,5000.0,4924.8,4980.0,0.0


## ALNS + set partitioning

In [27]:
if generate_df: 
    run_sp_name = f'{run_number}/sp/'
    run_sp_df = generate_run_df_alns(run_sp_name)
    run_sp_agg_df = aggregate_df_by_instance_group_alns(run_sp_df)

    run_sp_file_name = f'dataframes/performance/sp.pkl'
    run_sp_agg_file_name = f'dataframes/performance/sp_agg.pkl'
    run_sp_df.to_pickle(run_sp_file_name)
    run_sp_agg_df.to_pickle(run_sp_agg_file_name)

23-31-4-1 HAS LESS THAN FIVE SIMULATIONS!
25-33-4-1 HAS LESS THAN FIVE SIMULATIONS!
25-28-4-1 HAS LESS THAN FIVE SIMULATIONS!
23-30-4-1 HAS LESS THAN FIVE SIMULATIONS!
27-37-5-1 HAS LESS THAN FIVE SIMULATIONS!
27-32-5-1 HAS LESS THAN FIVE SIMULATIONS!
23 INSTANCE_SIZE LESS THAN FIVE INSTANCES!
25 INSTANCE_SIZE LESS THAN FIVE INSTANCES!
27 INSTANCE_SIZE LESS THAN FIVE INSTANCES!


In [28]:
run_sp_df = load_df('sp.pkl')
run_sp_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.8,0.8,0.8,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.8,0.9,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.3,1.3,1.3,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.2,1.1,1.2,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.1,1.8,1.9,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,1.9,1.7,1.8,5000,5000,5000.0,0.0
7,7-8-1-3,2080.2,2080.2,2080.2,2.1,1.9,2.0,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.8,4.2,4.5,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,2.9,2.2,2.6,5000,5000,5000.0,0.0


In [29]:
run_sp_agg_df = load_df('sp_agg.pkl')
run_sp_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,0.9,1.0,5000.0,5000.0,5000.0,0.0
1,7,2095.0,2095.0,2095.0,2.8,2.4,2.6,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,5.3,4.3,4.7,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,21.6,17.9,19.9,5000.0,5000.0,5000.0,0.0
4,13,3795.8,3970.5,3935.6,35.6,32.0,33.9,5000.0,5000.0,5000.0,0.0
5,15,8375.4,8392.0,8380.7,44.0,35.2,40.2,5000.0,5000.0,5000.0,0.7
6,17,4823.2,4983.7,4947.3,68.1,54.6,60.6,5000.0,5000.0,5000.0,1.2
7,19,4977.8,5160.7,5116.9,83.4,72.5,78.9,5000.0,5000.0,5000.0,1.6
8,21,9142.6,9324.6,9264.0,99.2,83.1,91.5,5000.0,5000.0,5000.0,2.4
9,23,5683.9,6065.3,4681.0,143.7,119.3,98.4,5000.0,5000.0,4000.0,1.2


## ALNS + local search + set partitioning

In [26]:
if generate_df:
    run_lssp_name = f'{run_number}/lssp/'
    run_lssp_df = generate_run_df_alns(run_lssp_name)
    run_lssp_agg_df = aggregate_df_by_instance_group_alns(run_lssp_df)

    run_lssp_file_name = f'dataframes/performance/lssp.pkl'
    run_lssp_agg_file_name = f'dataframes/performance/lssp_agg.pkl'
    run_lssp_df.to_pickle(run_lssp_file_name)
    run_lssp_agg_df.to_pickle(run_lssp_agg_file_name)

21-26-3-1 HAS LESS THAN FIVE SIMULATIONS!
21-28-3-1 HAS LESS THAN FIVE SIMULATIONS!
13-16-2-1 HAS LESS THAN FIVE SIMULATIONS!
15-15-2-1 HAS LESS THAN FIVE SIMULATIONS!
15-18-2-2 HAS LESS THAN FIVE SIMULATIONS!
17-18-3-1 HAS LESS THAN FIVE SIMULATIONS!
13 INSTANCE_SIZE LESS THAN FIVE INSTANCES!
17 INSTANCE_SIZE LESS THAN FIVE INSTANCES!
19 INSTANCE_SIZE LESS THAN FIVE INSTANCES!
21 INSTANCE_SIZE LESS THAN FIVE INSTANCES!


In [ ]:
run_lssp_df = load_df('lssp.pkl')
run_lssp_df

In [ ]:
run_lssp_agg_df = load_df('lssp_agg.pkl')
run_lssp_agg_df

## Exact solver

In [12]:
if generate_df:
    run_exact_name = 'results/'
    run_exact_df = generate_run_df_exact(run_exact_name)
    run_exact_agg_df = aggregate_df_by_instance_group_exact(run_exact_df)

    run_exact_file_name = f'dataframes/performance/exact.pkl'
    run_exact_agg_file_name = f'dataframes/performance/exact_agg.pkl'
    run_exact_df.to_pickle(run_exact_file_name)
    run_exact_agg_df.to_pickle(run_exact_agg_file_name)

In [13]:
run_exact_df = load_df('exact.pkl')
run_exact_df

,instance,obj,gap,preproc,model time,variables
0,5-5-1-1,2519.9,0.0,1.3,1.5,24808
1,5-5-1-2,2214.2,0.0,3.7,1.5,36406
2,5-6-1-1,1939.6,0.0,4.2,1.8,40610
3,5-7-1-1,2709.7,0.0,2.7,2.3,35034
4,5-7-1-2,1704.6,0.0,16.0,2.7,64346
5,7-8-1-1,2046.9,0.0,15.2,3.5,71164
6,7-8-1-2,1971.3,0.0,65.4,3.7,84072
7,7-8-1-3,2071.9,0.0,9.1,3.0,47880
8,7-9-1-1,2170.4,0.0,26.6,3.5,80958
9,7-9-1-2,2206.1,0.0,38.7,3.7,85362


In [14]:
run_exact_agg_df = load_df('exact_agg.pkl')
run_exact_agg_df

,instance_group,obj,gap,preproc,model time,variables
0,5,2217.6,0.0,5.6,2.0,40240.8
1,7,2093.3,0.0,31.0,3.5,73887.2
2,9,5627.5,0.0,43.7,5.7,124820.4
3,11,3450.2,0.0,2015.2,13.7,309915.6
4,13,3717.5,0.1,3493.6,20.3,447488.4
5,15,7257.5,0.1,3231.8,27.4,606759.0
6,Mean values,4060.6,0.0,1470.1,12.1,267185.2


## Parallel vs. sequential heuristics

In [ ]:
baseline_sequential_df = merge_dfs([run_baseline_df, run_sequential_df], True)
baseline_sequential_agg_df = merge_dfs([run_baseline_agg_df, run_sequential_agg_df], True)

In [ ]:
baseline_sequential_df

In [ ]:
baseline_sequential_agg_df

## ALNS vs. LNS

In [ ]:
baseline_lns_df = merge_dfs([run_baseline_df, run_lns_df], True)
baseline_lns_agg_df = merge_dfs([run_baseline_agg_df, run_lns_agg_df], True)

In [ ]:
baseline_lns_df

In [ ]:
baseline_lns_agg_df

## ALNS vs. ALNS + local search

In [ ]:
baseline_ls_df = merge_dfs([run_baseline_df, run_ls_df], True)
baseline_ls_agg_df = merge_dfs([run_baseline_agg_df, run_ls_agg_df], True)

In [ ]:
baseline_ls_df

In [ ]:
baseline_ls_agg_df

## ALNS vs. ALNS + set partitioning

In [ ]:
baseline_sp_df = merge_dfs([run_baseline_df, run_sp_df], True)
baseline_sp_agg_df = merge_dfs([run_baseline_agg_df, run_sp_agg_df], True)

In [ ]:
baseline_sp_df

In [ ]:
baseline_sp_agg_df

## ALNS vs. ALNS + local search + set partitioning

In [ ]:
baseline_lssp_df = merge_dfs([run_baseline_df, run_lssp_df], True)
baseline_lssp_agg_df = merge_dfs([run_baseline_agg_df, run_lssp_agg_df], True)

In [ ]:
baseline_lssp_df

In [ ]:
baseline_lssp_agg_df